This notebook is intended to show how to get biomasses from state variable data using the python code biomass.py

In [1]:
import numpy as np
import pandas as pd
import biomass as bm

In [2]:
# If ever lost, all code is documented. Call
?bm.biomass()
# OR for approximations
#?bm.biomass_approx()

Object `bm.biomass()` not found.


In [3]:
# How to use with a single dataset.
# Create a panda series with S, N, and E
test = pd.Series([100,10000,200000],index=['S','N','E'])
# Now simply call the biomass function
test_bm = bm.biomass(test)
# Can also call the analytic approximations
# First 0th order
test_bm0 = bm.biomass_approx(test,order=0) # calling order is optional here; order=0 by default
# Now 1st order
test_bm1 = bm.biomass_approx(test,order=1)

# Now print these
print("Biomass (numerical): {:.0f}".format(test_bm))
print("Biomass (0th order): {:.0f}".format(test_bm0))
print("Biomass (1st order): {:.0f}".format(test_bm1))

Biomass (numerical): 1380094
Biomass (0th order): 1623111
Biomass (1st order): 1405529


In [18]:
# Now how to use if we have a csv with state variable information
# Import the data
data = pd.read_csv('data_statevariables.csv')

In [20]:
d2 = data.drop(columns=['pB0','pB1','pBnum','lambda_1','lambda_2','Lat','Lon','Elev_m','beta',
                   'Notes1','Notes2','Notes3','Notes4','meas_est'])
cols = d2.columns.to_list()
cols2 = cols[:-1]
cols2.insert(11,cols[-1])


In [21]:
# This file will have only the state variables
d2[cols2].to_csv('data_statevariables_only.csv',index=False)

In [5]:
?bm.biomass_approx()

In [6]:
# Now add a column for numerical biomass data, and the approximations.
# The "p" indicates predicted, num is numerical, and 0 and 1 indicate the 0th and first order approximations.
# Note: Iterating through like this is actually bad practice for pandas, but this dataset is small, so it's fine
data['pBnum'] = np.zeros(len(data))
data['pBnum_2_3'] = np.zeros(len(data))
data['pB0'] = np.zeros(len(data))
data['pB1'] = np.zeros(len(data))
# Iterate through each row and append the biomass information
for index, row in data.iterrows():
    row23 = {'S': row['S'], 'N': row['N'], 'E': row['E_2_3']}
    data.loc[index,'pBnum_2_3'] = bm.biomass(row23,power=3/2)
    data.loc[index,'pBnum'] = bm.biomass(row)
    data.loc[index,'pB0'] = bm.biomass_approx(row)
    data.loc[index,'pB1'] = bm.biomass_approx(row,order=1)
    lambdas = bm.mete_lambdas(row)
    data.loc[index,'lambda_1'] = lambdas[0]
    data.loc[index,'lambda_2'] = lambdas[1]
    data.loc[index,'beta'] = lambdas.sum()
    # How are these lambdas doing? 
    #This should be very close to zero if the approximations in the underlying functions hold
    print(bm.constraints(lambdas,row))

[ 2.76117524e-13 -8.23106735e-12]
[ 2.11679295e-08 -3.50209982e-07]
[ 3.38172534e-10 -5.35741239e-09]
[ 2.21193183e-09 -4.27330519e-08]
[ 7.65126353e-06 -8.10650333e-05]
[ 2.79494437e-07 -4.00254769e-06]
[ 1.28905239e-14 -3.56512508e-16]
[-2.35209305e-16 -1.16826722e-16]
[1.55234724e-16 0.00000000e+00]
[-2.86364831e-16  0.00000000e+00]
[-3.57459898e-16  1.32168856e-16]
[-2.12390492e-16 -1.25476722e-16]
[1.76882990e-16 1.31240236e-16]
[1.57555128e-14 3.83997180e-15]
[ 3.73208522e-16 -1.37772712e-16]
[3.61396736e-16 1.37860119e-16]
[3.57505779e-16 0.00000000e+00]
[3.44199225e-14 4.73671602e-15]
[1.69594562e-15 1.50820843e-16]
[1.36110459e-16 2.01971203e-16]
[ 3.48855632e-16 -2.18085288e-16]
[-8.26319882e-14 -2.95866790e-13]
[2.52118912e-15 2.98515037e-15]
[-5.56948938e-15 -3.80384820e-15]
[1.04606954e-15 0.00000000e+00]
[ 4.21062362e-15 -1.26807234e-13]
[1.37319954e-16 0.00000000e+00]
[ 3.46052595e-09 -5.24135107e-08]
[ 2.79566269e-12 -6.76045226e-11]
[ 4.10074873e-14 -2.87656506e-13]
[ 

So the approximations are all holding well, though the Farewell Spit dataset has E not that much greater than N, and so the lambda calculations are not as precise (still within one percent). Additionally, for this dataset, the sum of lambda1 and lambda2 (beta) is less than zero, and so our approximation for the biomass equation does not work.

In [7]:
display(data)

Site  \
0          Point Reyes National Seashore Bishop Pines   
1          Point Reyes National Seashore Bishop Pines   
2                   Hubbard Brook Experimental Forest   
3                         Kellogg Biological Station    
4                         Kellogg Biological Station    
5                         Kellogg Biological Station    
6                     SCBI Large Forest Dynamics Plot   
7   CSIRO permanent rainforest plots of North Quee...   
8   CSIRO permanent rainforest plots of North Quee...   
9   CSIRO permanent rainforest plots of North Quee...   
10  CSIRO permanent rainforest plots of North Quee...   
11  CSIRO permanent rainforest plots of North Quee...   
12  CSIRO permanent rainforest plots of North Quee...   
13  CSIRO permanent rainforest plots of North Quee...   
14  CSIRO permanent rainforest plots of North Quee...   
15  CSIRO permanent rainforest plots of North Quee...   
16  CSIRO permanent rainforest plots of North Quee...   
17  CSIRO permanent rainforest plots of North Quee...   
18  CSIRO permanent rainforest plots of North Quee...   
19  CSIRO permanent rainforest plots of North Quee...   
20  CSIRO permanent rainforest plots of North Quee...   
21                                               RMBL   
22                                                BCI   
23                                             Cocoli   
24                                            Sherman   
25                                      UC Santa Cruz   
26                                             Catuba   
27                                     Harvard Forest   
28                                     Harvard Forest   
29                   Traunstein Forest Dynamics Plot    
30                           Atlantic Forest restinga   
31                           Atlantic Forest restinga   
32                           Atlantic Forest restinga   
33                           Atlantic Forest restinga   
34                           Atlantic Forest restinga   
35                           Atlantic Forest restinga   
36                Ordway-Swisher Forest Dynamics Plot   
37                                             Hawaii   
38                                             Hawaii   
39                                             Hawaii   
40                                             Hawaii   
41                                             Hawaii   

                       Plot                        Type             Location   \
0                Mt. Vision  Temperate forest community        California USA   
1                   Bayview  Temperate forest community        California USA   
2               Watershed 6             Temperate trees     New Hampshire USA   
3        Deciduous Forest 1             Temperate trees          Michigan USA   
4        Deciduous Forest 2             Temperate trees          Michigan USA   
5        Deciduous Forest 3             Temperate trees          Michigan USA   
6               Front Royal             Temperate trees          Virginia USA   
7                       EP3              Tropical trees  Queensland Australia   
8                      EP18              Tropical trees  Queensland Australia   
9                      EP19              Tropical trees  Queensland Australia   
10                     EP29              Tropical trees  Queensland Australia   
11                     EP30              Tropical trees  Queensland Australia   
12                     EP31              Tropical trees  Queensland Australia   
13                     EP32              Tropical trees  Queensland Australia   
14                     EP33              Tropical trees  Queensland Australia   
15                     EP34              Tropical trees  Queensland Australia   
16                     EP40              Tropical trees  Queensland Australia   
17                     EP41              Tropical trees  Queensland Australia   
18                     EP42              Tropical trees  Q

In [27]:
# To save
data.to_csv('data_statevariables.csv',index=False)

## Some goodness of prediction tests

In [21]:
# How close is this to data? Should be near 1 if we are getting the observed value.
print("Numerical")
ratio = data['pBnum']/data['B']
display(ratio)
print('Ratio: {:.2f} pm {:.2f}'.format(ratio.mean(),ratio.std()))
print()
print("Zeroth order")
ratio0 = data['pB0']/data['B']
display(ratio0)
print('Ratio: {:.2f} pm {:.2f}'.format(ratio0.mean(),ratio0.std()))
print()
print("First order")
ratio1 = data['pB1']/data['B']
display(ratio1)
print('Ratio: {:.2f} pm {:.2f}'.format(ratio1.mean(),ratio1.std()))
print()

Numerical


0     0.470988
1     1.266961
2     2.619427
3     1.213527
4     1.228977
5     1.171449
6     1.916850
7     1.094076
8     1.019964
9     1.040739
10    1.213038
11    1.106148
12    0.992462
13    0.961727
14    1.045652
15    1.046873
16    1.087485
17    1.191701
18    0.916161
19    1.061608
20    1.071910
21    1.100219
22    1.139860
23    0.786572
24    1.146325
25    1.577129
26    1.126499
27    1.698319
28    2.060959
29    2.814945
30    0.885891
31    0.726442
32    0.770833
33    0.807963
34    1.011287
35    0.668224
36    3.063694
37    1.183680
38    1.143643
39    1.192374
40    1.342518
41    1.167904
dtype: float64

Ratio: 1.24 pm 0.54

Zeroth order


0     0.539478
1     1.547698
2     3.318161
3     1.679629
4     1.604902
5     1.742658
6     2.018228
7     1.866382
8     1.875168
9     1.917834
10    2.029777
11    1.870499
12    1.884691
13    1.711425
14    1.745086
15    2.007593
16    1.742482
17    2.056947
18    1.873405
19    1.846363
20    1.814872
21    1.365088
22    1.192324
23    0.929637
24    1.289234
25    1.682569
26    1.918685
27    2.273167
28    2.307950
29    3.056991
30    8.870777
31    8.385651
32    3.270639
33    6.097507
34    6.274700
35    5.638191
36    2.909160
37    1.712571
38    1.626095
39    1.591414
40    1.835780
41    1.709316
dtype: float64

Ratio: 2.49 pm 1.83

First order


0     0.455755
1     1.219201
2     3.122682
3     1.183088
4     1.190549
5     1.155253
6     1.865673
7     1.125356
8     1.026044
9     1.082455
10    1.318957
11    1.162022
12    0.975051
13    1.070353
14    1.033821
15    1.028346
16    1.132237
17    1.231444
18    0.883087
19    1.061259
20    1.098062
21    1.060735
22    1.119179
23    0.763288
24    1.121975
25    1.530277
26    1.099399
27    1.888227
28    2.050340
29    2.846403
30   -0.108643
31   -0.296881
32    0.566217
33    0.163419
34    0.534050
35    0.036996
36    2.786405
37    1.149196
38    1.143392
39    1.218773
40    1.328248
41    1.162281
dtype: float64

Ratio: 1.16 pm 0.68



## Some goodness of approximation tests

In [13]:
# For each data, how good is the zeroth order and first order prediction compared to the numerical test?
# Should be near 0 for being exact with the numerical answer.
print("Zeroth order")
frac0 = (data['pB0']-data['pBnum'])/data['pBnum']
display(frac0)
print('Number of datasets where we are within 10%: {}/{}'.format(np.sum((frac0<0.1)),len(frac0)))
print()
print('Percent: {:.2f} pm {:.2f}'.format(frac0.mean(),frac0.std()))
print()
print("First order")
frac1 = (data['pB1']-data['pBnum'])/data['pBnum']
display(frac1)
print('Number of datasets where we are within 10%: {}/{}'.format(np.sum((frac1<0.1)),len(frac1)))
print()
print('Percent: {:.2f} pm {:.2f}'.format(frac1.mean(),frac1.std()))
print()

Zeroth order


0      0.145418
1      0.221583
2      0.266751
3      0.384089
4      0.305885
5      0.487609
6      0.052888
7      0.705899
8      0.838465
9      0.842762
10     0.673300
11     0.691003
12     0.899006
13     0.779532
14     0.668897
15     0.917705
16     0.602304
17     0.726061
18     1.044842
19     0.739214
20     0.693119
21     0.240743
22     0.046027
23     0.181883
24     0.124667
25     0.066856
26     0.703228
27     0.338481
28     0.119842
29     0.085986
30     9.013392
31    10.543450
32     3.242991
33     6.546769
34     5.204670
35     7.437574
36    -0.050440
37     0.446819
38     0.421855
39     0.334661
40     0.367416
41     0.463575
dtype: float64

Number of datasets where we are within 10%: 5/42

Percent: 1.39 pm 2.51

First order


0    -0.032342
1    -0.037697
2     0.192124
3    -0.025083
4    -0.031268
5    -0.013826
6    -0.026699
7     0.028591
8     0.005961
9     0.040083
10    0.087317
11    0.050512
12   -0.017543
13    0.112948
14   -0.011315
15   -0.017697
16    0.041151
17    0.033350
18   -0.036101
19   -0.000328
20    0.024398
21   -0.035887
22   -0.018143
23   -0.029602
24   -0.021242
25   -0.029707
26   -0.024057
27    0.111821
28   -0.005153
29    0.011175
30   -1.122637
31   -1.408678
32   -0.265448
33   -0.797740
34   -0.471910
35   -0.944635
36   -0.090508
37   -0.029133
38   -0.000220
39    0.022140
40   -0.010629
41   -0.004815
dtype: float64

Number of datasets where we are within 10%: 39/42

Percent: -0.11 pm 0.34

